### Initializing connection

In [1]:
import pyodbc

server = 'ICHIM\SQLEXPRESS03'
database = 'TicketManagementSystem'
username = ''
password = ''

connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

conn = pyodbc.connect(connection_string)

### Populating Users table with names from data/names/US.csv
Made up email through combination of first_last names + random @, with random age <br/>
Password random amount of length, last 2 always numbers <br/>
Conjured up a salt for each, used SHA2_256 hashing method <br/>

In [ ]:
cursor = conn.cursor()
cursor.execute('SELECT * FROM Users')

column_names = [column[0] for column in cursor.description]
print(column_names)

results = cursor.fetchall()
for row in results[:3]:
    print(row)

#### Data reading

In [72]:
import pandas as pd

# Names from the United States
us_names_path = 'data/names/US.csv'
column_names = ['First Name', 'Last Name', 'Gender', 'Country']

df = pd.read_csv(us_names_path, names=column_names)

#### Filtering + Save

In [ ]:
import random
import re
import uuid
import string
import hashlib
import os
import csv

def has_arabic_characters(text):
    # if reading wrong from dataset, happens!
    if(type(text)) == float:
        return True
    
    arabic_pattern = re.compile(r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF]')
    return bool(arabic_pattern.search(text))

def generate_password(length):
    letter_sequence = ''
    for _ in range(length - 2):
        letter_sequence += random.choice(string.ascii_letters)

    number_sequence = ''
    for _ in range(2):
        number_sequence += str(random.randint(0, 9))
    return letter_sequence + number_sequence

def has_nan(user):
    if type(user['ID']) != uuid.UUID:
        return True
    if type(user['Email']) != str:
        return True
    if type(user['PasswordHash']) != str:
        return True
    if type(user['Salt']) != uuid.UUID:
        return True
    if type(user['Sex']) != str:
        return True
    if type(user['Age']) != int:
        return True
    if type(user['IsCustomer']) != int:
        return True
    

def in_table(email, cursor):
    select_query = f"SELECT * FROM Users WHERE Email = ?"
    cursor.execute(select_query, (email,))
    rows = cursor.fetchall()
    if rows:
        # entry found
        return True
    return False

def save_user(user, conn, cursor):
    if has_nan(user) or in_table(user['Email'], cursor):
        return
    
    insert_query = f"INSERT INTO Users (ID, Email, PasswordHash, Salt, Sex, Age, IsCustomer) VALUES (?, ?, ?, ?, ?, ?, ?)"
    cursor.execute(insert_query, (
        user['ID'], user['Email'], user['PasswordHash'], user['Salt'], user['Sex'], user['Age'], user['IsCustomer']
    ))
    conn.commit()


user_credentials_list = []
emails = ['@gmail.com', '@yahoo.com', '@hotmail.com', '@outlook.com']
print(['ID', 'Email', 'PasswordHash', 'Salt', 'Sex', 'Age', 'IsCustomer'])
for index, row in df[10005:20000].iterrows():
    first_name = row['First Name']
    last_name = row['Last Name']

    # filter out arabic names
    if has_arabic_characters(first_name) == False and has_arabic_characters(last_name) == False:
        gender = row['Gender']
        age = random.randint(5, 65)

        random_email_finisher = random.choice(emails)
        email = first_name + '_' + last_name + random_email_finisher

        salt = uuid.uuid4()

        password_length = random.randint(5, 15)
        password = generate_password(15)
        password_hash = hashlib.sha256(password.encode() + str(salt).encode()).hexdigest()

        IsCustomer = 1
        
        ID = uuid.uuid4()

        user_credentials = [email, password]
        user_credentials_list.append(user_credentials)

        user = {
            'ID': ID,
            'Email' : email,
            'PasswordHash' : password_hash,
            'Salt' : salt,
            'Sex' : gender,
            'Age' : age,
            'IsCustomer' : IsCustomer
        }

        save_user(user, conn, cursor)

save_credentials_path = 'data/credentials/user_credentials.csv'
with open(save_credentials_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow((['Email', 'Password']))
    print(user_credentials_list)
    writer.writerows(user_credentials_list)
        

### Populating Venues table with bigger variety of types

In [ ]:
cursor = conn.cursor()
cursor.execute('SELECT * FROM Venues')

column_names = [column[0] for column in cursor.description]
print(column_names)

results = cursor.fetchall()
for row in results[:3]:
    print(row)

In [139]:
venue_types = [
    'Conference Center',
    'University',
    'Ballroom',
    'Theater'
]
capacities = [
    500, 800, 100, 350, 200, 400, 450, 300, 250, 1000, 850, 870
]

for i in range(len(venue_types)):
    print(venue_types[i] + " " + str(capacities[i]))
    


Conference Center 500
University 800
Ballroom 100
Theater 350


In [ ]:
def save_venue(venue, conn, cursor):
    insert_query = f"INSERT INTO Venues (ID, Location, Type, Capacity, PricePerHour) VALUES (?, ?, ?, ?, ?)"
    cursor.execute(insert_query, (
        venue['ID'], venue['Location'], venue['Type'], venue['Capacity'], venue['PricePerHour']
    ))
    conn.commit()

locations = ['Cluj-Napoca', 'Iasi', 'Bucuresti', 'Salzburg', 'Zurich', 'Geneva', 'Venetia', 'Berlin', 'London', 'Yorkshire', 'Liverpool',
             'Timisoara', 'Munich', 'Dublin', 'Paris', 'Lyon', 'Barcelona', 'Sevilla']
prices_per_hour = [
    100.30, 180.20, 80.10, 50.99, 51.00, 55.00, 80.00, 25.99, 28.00
]
for i in range(50):
    venue = {
        'ID' : uuid.uuid4(),
        'Location' : random.choice(locations),
        'Type' : random.choice(venue_types),
        'Capacity' : random.choice(capacities),
        'PricePerHour' : random.choice(prices_per_hour)
    }
    save_venue(venue, conn, cursor)

### Populating EventTypes table with bigger variety of types

In [ ]:
cursor = conn.cursor()
cursor.execute('SELECT * FROM EventTypes')

column_names = [column[0] for column in cursor.description]
print(column_names)

results = cursor.fetchall()
for row in results[:3]:
    print(row)

In [148]:
conference_center_event_names = [
    'Convention', 'Seminar', 'Workshop', 'Expo', 'Business networking'
]

ballroom_event_names = [
    'Wedding', 'Gala', 'Award Ceremony', 'Fashion Show'
]

theater_event_names = [
    'Play', 'Musical', 'Opera', 'Dance Performance', 'Stand-up Performance', 'Comedy-Show'
]

university_event_names = [
    'Academic Lecture', 'Presentation', 'Guest Speaker', 'Student Performance', 'Training Program', 'Graduation Ceremony'
]

In [151]:
def save_event_names(event_names, conn, cursor):
    for event_name in event_names:
        insert_query = f"INSERT INTO EventTypes (ID, Name) VALUES (?, ?)"
        cursor.execute(insert_query, (
            uuid.uuid4(), event_name    
        ))
        conn.commit()
save_event_names(conference_center_event_names, conn, cursor)
save_event_names(ballroom_event_names, conn, cursor)
save_event_names(theater_event_names, conn, cursor)
save_event_names(university_event_names, conn, cursor)

### Populating Events table

In [ ]:
import datetime

cursor = conn.cursor()
cursor.execute('SELECT * FROM EventTypes')

column_names = [column[0] for column in cursor.description]

years = [
    2018, 2019, 2020, 2021, 2022, 2023
]

def get_dates(year):
    month = random.randint(1, 12)
    start_day = random.randint(1, 24)
    end_day = start_day + random.randint(1, 4)

    start_date = datetime.datetime(year, month, start_day)
    end_date = datetime.datetime(year, month, end_day)
    
    return start_date, end_date

# description si name nu am destula imaginatie x)
def save_event(ID_eventtype, event_type, description, name):
    query = "SELECT * FROM Venues WHERE Type = ?"
    cursor.execute(query, (event_type))
    venues = cursor.fetchall()

    for _ in range(random.randint(5, 15)):
        for year in years:
            start_date, end_date = get_dates(year)
            random_venue = random.choice(venues)

            # print(ID_eventtype, random_venue, start_date, end_date)
            print(random_venue[0], ID_eventtype, description, name, start_date, end_date)

            insert_query = "INSERT INTO Events (ID, Venue_ID, EventType_ID, Description, Name, StartDate, EndDate) VALUES (?, ?, ?, ?, ?, ?, ?)"
            cursor.execute(insert_query,
                        (uuid.uuid4(), random_venue[0], ID_eventtype, description, name, start_date, end_date)
            )
            conn.commit()

event_types = cursor.fetchall()
for row in event_types:
    ID_eventtype = row[0]
    event_name = row[1]
    if event_name in conference_center_event_names:
        save_event(ID_eventtype, 'Conference Center', event_name, event_name)
    if event_name in ballroom_event_names:
        save_event(ID_eventtype, 'Ballroom', event_name, event_name)
    if event_name in theater_event_names:
        save_event(ID_eventtype, 'Theater', event_name, event_name)
    if event_name in university_event_names:
        save_event(ID_eventtype, 'University', event_name, event_name)

### Populating TicketCategories table

In [ ]:
cursor = conn.cursor()
cursor.execute('SELECT * FROM Events')

events = cursor.fetchall()

vip_prices = [
    1000, 1100, 1200, 1300, 1400, 1500, 1150, 1250, 1350, 2000, 2500
]

standard_prices = [
    100, 200, 300, 400, 500, 700, 150, 250, 350, 450
]

def save_ticket(Event_ID, description, price, cursor, conn):
    print(description, price)
    insert_query = "INSERT INTO TicketCategories (ID, Event_ID, Description, Price) VALUES (?, ?, ?, ?)"
    
    cursor.execute(insert_query,
        (uuid.uuid4(), Event_ID, description, price)
    )   
    conn.commit()


for event in events:
    if random.random() > 0.4:
        save_ticket(event[0], 'VIP', random.choice(vip_prices), cursor, conn)
    save_ticket(event[0], 'Standard', random.choice(standard_prices), cursor, conn)

### Populating Orders table

In [14]:
import random
import datetime
import uuid
cursor = conn.cursor()
cursor.execute('SELECT * FROM Users')

users = cursor.fetchall()

def save_order(user_id, ticket_description, cursor, conn):
    query = F"SELECT * FROM TicketCategories WHERE Description = ?"
    cursor.execute(query, (ticket_description,))

    ticketcategories = cursor.fetchall()
    for no_tickets in range(random.randint(5, 10)):
        ticket = random.choice(ticketcategories)

        NumberOfTickets = random.randint(3, 10)
        TotalPrice = NumberOfTickets * ticket[3]

        insert_query = F"INSERT INTO Orders (ID, User_ID, TicketCategories_ID, OrderedAt, NumberOfTickets, TotalPrice) VALUES (?, ?, ?, ?, ?, ?)"
        cursor.execute(insert_query,
            (uuid.uuid4(), user_id, ticket[0], datetime.datetime.now(), NumberOfTickets, TotalPrice)
        )
        conn.commit()

for user in users[3:]:
    if random.random() < 0.7:
        save_order(user[0], 'Standard', cursor, conn)
    else:
        save_order(user[0], 'VIP', cursor, conn)